In [1]:
import pandas as pd
import numpy as np

In [2]:
# split to hour, minute, second

def add_time_string(df):
    df = df.reset_index(drop=True)
    df["queryts_new"] = pd.DataFrame(pd.to_datetime(df.queryts, unit='s'))
    df.queryts_new = df.queryts_new.astype(str)

    con = pd.DataFrame(df.queryts_new.str.split(" ",2).tolist(), columns = ['a','b'])
    con1 = pd.DataFrame(con.b.str.split(":",3).tolist(), columns = ['hr','min','sec'])
    df["hr"] = con1.hr
    df.hr = pd.to_numeric(df.hr, errors = 'coerce')

    print (df.shape)
    return df

In [3]:
# 每個檔案(出現時段平均, 出現時段標準差, 出現時段唯一數量)

def groupby_file_hr(df_val):
    f = {'hr': ['mean', 'std', lambda x: len(set(x))]}

    g = df_val.groupby(["file_id"]).aggregate(f).reset_index()
    g.columns = ["file_id", "file_id_hr_mean", "file_id_hr_std", "file_id_hr_count"]
    g.file_id_hr_std = g.file_id_hr_std.fillna(0)

    print (g.shape)
    return g

In [4]:
# 一天每小時的檔案數量(平均, 標準差, 極大值, 極小值)
# 一天每小時的顧客X商品唯一數量(平均, 標準差, 極大值, 極小值)

def two_minus_one(x):
    x = list(x)
    if len(x) > 1:
        m_ = x[1] - x[0]
    else:
        m_ = x[0] - x[0]
    return m_

def groupby_file_hrcount_perday(df_val):
    # step 1: 算出(file_id, custXprod_id, hour)的(數量, 顧客商品對)的數量
    f = {'custXprod_id': ['count', lambda x: len(set(x))]}
    g1 = df_val.groupby(["file_id", "date_new", "hr"]).aggregate(f).reset_index()
    g1.columns = ["file_id", "date_new", "hr",
                  "file_id_hr_count", "custXprod_id_hr_count"]
    g1["custXprod_id_hr_count_file_id_hr_count_ratio"] = g1.custXprod_id_hr_count / g1.file_id_hr_count

    # step 2: 算出(file_id, custXprod_id)在每一天，小時的(平均, 標準差, 極大值, 極小值)
    col_g2 = ["file_id", "date_new"]
    for varible_ in g1.columns[3:]:
        for stat in ["_mean", "_std", "_max", "_min"]:
            col_g2.append(varible_ + stat)

    f_ = ['mean', 'std', 'max', 'min']
    f1 = {g1.columns[3]: f_, g1.columns[4]: f_, g1.columns[5]: f_}

    g2 = g1.groupby(['file_id', 'date_new']).aggregate(f1).reset_index()
    g2.columns = col_g2
    g2 = g2.fillna(0)

    f2 = lambda x: x - min(x)
    g2_1 = g2.groupby('file_id')["date_new"].apply(f2).reset_index(drop = True)
    g2["date_perfile"] = g2_1

    # step 3: 列出第一天(file_id, custXprod_id)在第一天，小時的(平均, 標準差, 極大值, 極小值)
    g3_col = ['file_id']
    for d in range(1):
        for varible_ in g2.columns[2:14]:
            g3_col.append(varible_ + "_d" + str(d))

    g3 = g2[(g2.date_perfile == 0)]
    g3 = g3.drop(['date_new', 'date_perfile'], axis=1)
    g3.columns = g3_col
    print ("shape of dataframe about Hour1 :", g3.shape)

    # step 4: 最大流量每小時
    f1 = {"file_id_hr_count": 'max',
          "custXprod_id_hr_count": 'max'}

    g = g1.groupby(['file_id']).aggregate(f1).reset_index()
    g.columns = ['file_id', 'file_id_hr_count_max', 'custXprod_id_hr_count_max']
    print ("shape of dataframe about Hour2 :", g.shape)

    # step 5: 利用step2得到的資料集算出一些資訊(mean, max_minus_min, two_minus_one)
    col_ = g2.columns
    col_g4_1 = ["file_id"]
    for i in [2,3,6,7,10,11]:
        col_g4_1.append(col_[i]+"_mean")
        col_g4_1.append(col_[i]+"_mmm")
    f_ = ['mean', lambda x: max(x) - min(x)]
    f3 = {col_[2]: f_, col_[3]: f_, col_[6]: f_, col_[7]: f_, col_[10]: f_, col_[11]: f_}
    g4_1 = g2.groupby(['file_id']).agg(f3).reset_index()
    g4_1.columns = col_g4_1

    col_g4_2 = ["file_id"]
    for i in [2,3,6,7,10,11]:
        col_g4_2.append(col_[i]+"_tmo")

    f_ = [lambda x: two_minus_one(x)]
    f4 = {col_[2]: f_, col_[3]: f_, col_[6]: f_, col_[7]: f_, col_[10]: f_, col_[11]: f_}
    g4_2 = g2.groupby(['file_id']).agg(f4).reset_index()
    g4_2.columns = col_g4_2

    g4 = pd.merge(g4_1, g4_2, how="left", on="file_id")
    print ("shape of dataframe about Hour3 :", g4.shape)

    g = pd.merge(g, g3, how = "left", on = "file_id")
    g = pd.merge(g, g4, how = "left", on = "file_id")
    return g

In [5]:
df = pd.DataFrame()
for file_ in ["train1","train2","train3","valid"]:
    df_ = pd.read_pickle("dataset/" + str(file_) + ".pkl")
    df = pd.concat([df, df_], axis=0)
    
df = add_time_string(df)
df['custXprod_id'] = df.customer_id + df.product_id

g_train = groupby_file_hr(df)
g2 = groupby_file_hrcount_perday(df)

g_train = pd.merge(g_train, g2, how = "left", on = "file_id")
print (g_train.shape)

(54250245, 9)
(52518, 4)
shape of dataframe about Hour1 : (52518, 13)
shape of dataframe about Hour2 : (52518, 3)
shape of dataframe about Hour3 : (52518, 19)
(52518, 36)


In [6]:
df = pd.DataFrame()
for file_ in ["test_1","test_2","test_3"]:
    df_ = pd.read_pickle("dataset/" + str(file_) + ".pkl")
    df = pd.concat([df, df_], axis=0)

df = add_time_string(df)
df['custXprod_id'] = df.customer_id + df.product_id

g_test = groupby_file_hr(df)
g2 = groupby_file_hrcount_perday(df)

g_test = pd.merge(g_test, g2, how = "left", on = "file_id")
print (g_test.shape)

(29022865, 9)
(29376, 4)
shape of dataframe about Hour1 : (29376, 13)
shape of dataframe about Hour2 : (29376, 3)
shape of dataframe about Hour3 : (29376, 19)
(29376, 36)


In [7]:
g = pd.concat([g_train, g_test], axis=0)

g.to_csv("dataset/hour.csv", index=False)

In [8]:
print (g.shape)
g.iloc[0, :]

(81894, 36)


file_id                                                   0000e2398b12121a85166fed5fe2a3da
file_id_hr_mean                                                                    8.04255
file_id_hr_std                                                                     1.26761
file_id_hr_count                                                                         5
file_id_hr_count_max                                                                    38
custXprod_id_hr_count_max                                                                1
file_id_hr_count_mean_d0                                                             10.75
file_id_hr_count_std_d0                                                            18.1728
file_id_hr_count_max_d0                                                                 38
file_id_hr_count_min_d0                                                                  1
custXprod_id_hr_count_mean_d0                                                            1